In [ ]:
# Basic notebook to fetch the mhtml of a page and add it to example data

In [38]:
import os
import urllib.parse
import uuid

from playwright.async_api import async_playwright

In [39]:
# Setup a playwright instance. We need to do this to access the dev tool functions
p = await async_playwright().__aenter__()
browser = await p.chromium.launch(headless=False)
context = await browser.new_context()
page = await context.new_page()
client = await context.new_cdp_session(page)

In [44]:
# Go to your desired URL
desired_url = "https://c3.ai/job-description/?gh_jid=6181786002"
await page.goto(desired_url)

<Response url='https://jobs.careers.microsoft.com/global/en/job/1658879/Software-Engineering-II' request=<Request url='https://jobs.careers.microsoft.com/global/en/job/1658879/Software-Engineering-II' method='GET'>>

In [45]:
def get_file_path(url: str) -> str:
    parsed_url = urllib.parse.urlparse(url)
    domain = parsed_url.netloc
    domain = domain.replace("www.", "").replace(".com", "")
    return f"./{domain}.mhtml"

In [46]:
# Capture page content as MHTML
result = await client.send('Page.captureSnapshot', {'format': 'mhtml'})
mhtml = result['data']

# Create example values
values = {
    "id": str(uuid.uuid4()),
    "url": desired_url,
    "source": "mhtml",
    "domain": "UPDATE",
    "subdomain": "UPDATE",
    "type": "fetch",
    "goal": {},
    "evals": [
      {
        "type": "json_match",
        "expected": {}
      }
    ]
  }

# Write MHTML content to the specified file
folder_path  = f"./python/bananalyzer/data/{values['id']}"
os.makedirs(folder_path, exist_ok=True)
file_path = os.path.join(folder_path, "index.mhtml")
with open(file_path, 'w') as f:
    f.write(mhtml)

# Print the path to the console
print(f'Page saved as MHTML at: {file_path}')
file_path = os.path.join(folder_path, "index.mhtml")
with open(file_path, 'w') as f:
    f.write(mhtml)

# Print the path to the console
print(f'Page saved as MHTML at: {file_path}')

# Save values to JSON file
import json

json_file_path = "./python/bananalyzer/data/examples.json"
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

data.append(values)

with open(json_file_path, 'w') as json_file:
    json.dump(data, json_file, indent=4)
    json_file.write('\n')
# print(json.dumps(data, indent=4))


Page saved as MHTML at: /Users/asimshrestha/Documents/GitHub/bananalyzer/python/bananalyzer/data/ad142573-4dcb-4c1e-9a4d-6fa1fa818026/index.mhtml
Page saved as MHTML at: /Users/asimshrestha/Documents/GitHub/bananalyzer/python/bananalyzer/data/ad142573-4dcb-4c1e-9a4d-6fa1fa818026/index.mhtml
